<a href="https://colab.research.google.com/github/Mafia3641/Collab_practice/blob/main/Practice_0_8_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
def request(url):
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()

    return response.status_code


def get_apod(api_key):
    r = request(f"https://api.nasa.gov/planetary/apod?api_key={api_key}")

    title = r['title']
    date = r['date']
    expl = r['explanation']
    cr = r['copyright']

    print("Title: ", title)
    print("Date: ", date)
    print("Copyright: ", cr)
    print("Explanation: ", expl)



def get_photos(api_key):
    date = input("Enter date format: dd.mm.year: ")

    if date.count('.') == 3:
        date = date[:-1]
    if date.count('/') == 2 or date.count('-') == 2:
        date = date.replace('/', '.')
        date = date.replace('-', '.')
    if date.count('.') == 2:
        date = date.split('.')
        date = list(map(int, date))
    else:
        print("Wrong interrupt! ")
        return


    print('Choose camera type from the list and write down its name')
    print('FHAZ')
    print('RHAZ')
    print('MAST')
    print('CHEMCAM')
    print('MAHLI')
    print('MARDI')
    print('MARDI')
    print('NAVCAM')
    print('PANCAM')
    print('MINITES')

    cam_type = input("Camera type: ").upper()
    if cam_type in ['FHAZ', "RHAZ", "MAST", "CHEMCAM", "MAHLI", "MARDI", "NAVCAM", "PANCAM", "MINITES"]:
        url = 'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos'

        params = {
            'earth_date': date,
            'camera': cam_type,
            'api_key': api_key
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            photos = response.json().get('photos', [])
            return [photo['img_src'] for photo in photos] if photos else "Фотографии не найдены."
        else:
            return "Ошибка при получении фотографий с марсохода."
    else:
        print("Wrong interrupt! ")
        return 0


def get_near_objects(api_key):
    date = input("Введите дату в формате dd.mm.year: ").split('.')
    dist = int(input("Введите размер объекта: "))
    day, month, year = date
    date = f"{year}-{month}-{day}"
    params = {
        'start_date': date,
        'end_date': date,
        'api_key': api_key
    }
    r = requests.get("https://api.nasa.gov/neo/rest/v1/feed", params=params)


    neos = r.json().get('near_earth_objects', {}).get(date, [])
    if neos:
        result = []
        for neo in neos:
            result.append(
                f"Название: {neo['name']}, Размер: {neo['estimated_diameter']['meters']['estimated_diameter_max']} м, Опасность: {'Да' if neo['is_potentially_hazardous_asteroid'] else 'Нет'}")
        return "\n".join(result)
    else:
        return "Объекты не найдены."


def get_weather(api_key):
    r = request(f'https://api.nasa.gov/DONKI/FLR?api_key={api_key}')

    if r:
        result = []

        for fl in r:
            result.append(f"Дата начала: {fl['beginTime']}, Класс: {fl['classType']}")
        return "\n".join(result)
    else:
        return "Данные о солнечных вспышках не найдены."


def main():
    api_key = "[ENTER YOUR API HERE]"

    print("Выберете запрос из списка: ")
    print("1) Астрономическая картинка дня ")
    print("2) Фотографии с марсохода ")
    print("3) Объекты, сближающиеся с землей ")
    print("4) Данные о погоде ")
    question = int(input("Введите номер запроса: "))

    if question == 1:
        get_apod(api_key)
    elif question == 2:
        print(get_photos(api_key))
    elif question == 3:
        print(get_near_objects(api_key))
    elif question == 4:
        print(get_weather(api_key))
    else:
        print("Wrong interrupt! ")


main()

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests

BASE_URL = "https://api.artic.edu/api/v1/artworks"


def fetch_artworks(page=1):
    url = f"{BASE_URL}?page={page}&limit=10&fields=id,title,artist_display,date_display"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        artworks = data.get("data", [])
        pag = data.get("pagination", {})
        return artworks, pag

    else:
        print("Ошибка! ")
        return [], {}


def filter_artworks_by_artist(artist_name, page=1):
    artworks, pagination = fetch_artworks(page)
    filtered_artworks = [art for art in artworks if artist_name.lower() in art.get("artist_display", "").lower()]

    if not filtered_artworks:
        print("Произведения искусства с указанным художником не найдены.")

    return filtered_artworks, pagination


def display_artwork_details(artwork_id):
    url = f"{BASE_URL}/{artwork_id}"
    response = requests.get(url)

    if response.status_code == 200:
        art = response.json().get("data", {})
        details = f"Название: {art.get('title')}\n" \
                  f"Художник: {art.get('artist_display')}\n" \
                  f"Дата: {art.get('date_display')}\n" \
                  f"Место: {art.get('place_of_origin')}\n" \
                  f"Техника: {art.get('medium_display')}\n" \
                  f"Ссылка: {art.get('image_id')}"
        print(details)

    else:
        print("Ошибка при получении данных о произведении искусства.")


def main():
    page = 1

    while True:
        print("")
        print("1. Показать список произведений искусства")
        print("2. Фильтровать произведения по имени художника")
        print("3. Показать подробную информацию о работе")
        print("4. Перейти на следующую страницу")
        print("5. Перейти на предыдущую страницу")
        print("6. Выйти")

        choice = input("Выберите действие: ")

        if choice == '1':
            artworks, pagination = fetch_artworks(page)

            for art in artworks:
                print(f"ID: {art['id']}, Название: {art['title']}, Художник: {art['artist_display']}")

        elif choice == '2':
            artist_name = input("Введите имя художника для фильтрации: ")
            artworks, pagination = filter_artworks_by_artist(artist_name, page)

            for art in artworks:
                print(f"ID: {art['id']}, Название: {art['title']}, Художник: {art['artist_display']}")

        elif choice == '3':
            artwork_id = input("Введите ID произведения для просмотра деталей: ")
            display_artwork_details(artwork_id)

        elif choice == '4':
            if pagination.get("current_page", 1) < pagination.get("total_pages", 1):
                page += 1
                print(f"Переход на страницу {page}")

            else:
                print("Это последняя страница.")

        elif choice == '5':
            if page > 1:
                page -= 1
                print(f"Переход на страницу {page}")

            else:
                print("Это первая страница.")

        elif choice == '6':
            print("Выход из программы.")
            break

        else:
            print("Wrong interrupt! ")


main()


# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
import requests
from datetime import datetime, timedelta

URL = "https://api.coingecko.com/api/v3"
portfolio = {}


def get_price(coin_id, currency="usd"):
    url = f"{URL}/simple/price"
    params = {
        "ids": coin_id,
        "vs_currencies": currency,
        "include_24hr_change": "true"
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data.get(coin_id, {})


def add_to_portfolio(coin_id, amount):
    if coin_id in portfolio:
        portfolio[coin_id] += amount

    else:
        portfolio[coin_id] = amount


def get_portfolio_value(currency="usd"):

    total_value = 0

    for coin_id, amount in portfolio.items():
        price_info = get_price(coin_id, currency)
        price = price_info.get(currency, 0)
        total_value += price * amount
    return total_value


def track_price_changes():
    for coin_id in portfolio:
        price_info = get_price(coin_id)
        change_24h = price_info.get("usd_24h_change", 0)
        print(f"{coin_id.capitalize()}: изменение за 24ч: {change_24h:.2f}%")


def get_historical_data(coin_id, days=7, currency="usd"):

    url = f"{URL}/coins/{coin_id}/market_chart"
    params = {"vs_currency": currency, "days": days}

    response = requests.get(url, params=params)
    data = response.json()
    prices = data.get("prices", [])

    for timestamp, price in prices:
        date = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')
        print(f"{date}: {price:.2f} {currency}")


def main():
    while True:
        print("\nМеню:")
        print("1. Добавить криптовалюту в портфель")
        print("2. Показать текущую стоимость портфеля")
        print("3. Отслеживать изменения цен в портфеле за последние 24ч")
        print("4. Получить исторические данные по криптовалюте")
        print("5. Выйти")

        choice = input("Выберите действие: ")

        if choice == '1':
            coin_id = input("Введите ID криптовалюты (например, 'bitcoin'): ").strip().lower()
            amount = float(input("Введите количество: "))
            add_to_portfolio(coin_id, amount)
            print(f"{amount} {coin_id} добавлено в портфель.")

        elif choice == '2':
            currency = input("Введите валюту (например, 'usd'): ").strip().lower()
            total_value = get_portfolio_value(currency)
            print(f"Общая стоимость портфеля: {total_value:.2f} {currency}")

        elif choice == '3':
            track_price_changes()

        elif choice == '4':
            coin_id = input("Введите ID криптовалюты: ").strip().lower()
            days = int(input("Введите количество дней (например, 7): "))
            currency = input("Введите валюту (например, 'usd'): ").strip().lower()
            get_historical_data(coin_id, days, currency)

        elif choice == '5':
            print("Выход из программы.")
            break

        else:
            print("Wrong interrupt! ")


main()


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код